# Create batch

## Set correct directory

In [1]:
cd ..

/bptk-py


## Import libraries

In [2]:
import pandas as pd
import csv
import time
import pickle
from elasticsearch import Elasticsearch
import os.path
from src.config.conf import width,height
from src.setup import setup_model
from ipywidgets import widgets
from ipywidgets import IntSlider

## Calculate Batch Results

In [ ]:
es = Elasticsearch([{'host': 'es_node1', 'port': 9200}])


car_list=["1_car","2_car","3_car"]
keys=['id','time_start','time_end','revenue','cost','profit']

data={"time":0}

for car in car_list:
    data[car]={
        "sum_revenue":0,
        "sum_cost":0,
        "profit":0
    }
    
# initiale the data pickle

with open("csv/data.pickle",'wb') as data_file_obj:
    pickle.dump(data,data_file_obj)   

sleep_time=60
time_delta=1440
last_batch_processing_time=0

while True:
    
    
    # read the time
    time_file_object = open("csv/sim_time.pickle",'rb')  
    current_time = pickle.load(time_file_object)

    if current_time>=last_batch_processing_time+time_delta:
        
        # read the data
        data_file_object = open ("csv/data.pickle",'rb')
        data=pickle.load(data_file_object)
        data["time"]=last_batch_processing_time+time_delta
    
        for car in car_list:
            time_from=last_batch_processing_time+1
            time_to=last_batch_processing_time+time_delta+1
               
            profit_result=es.search(index=car,body={"size": 0,
                "aggs" : {
                    "sum_profit" : {
                        "date_range": {
                            "field": "time",
                            "ranges": [
                                { "from": time_from,
                                  "to": time_to } 
                            ]
                        },
                        "aggs": {
                            "sum_cost": {
                              "sum": {
                                "field": "cost"
                              }
                            },
                            "sum_revenue":{
                              "sum":{
                                "field":"revenue"
                              }
                            },
                            "profit": {
                                "bucket_script": {
                                    "buckets_path": {
                                      "sumRev": "sum_revenue",
                                      "sumCost": "sum_cost"
                                    },
                                    "script": "params.sumRev - params.sumCost"
                                }
                            }
                        }
                    }
                }})

            for result in ["sum_revenue","sum_cost","profit"]:
                if result in profit_result["aggregations"]["sum_profit"]["buckets"][0].keys():
                    data[car][result]+=(profit_result['aggregations']['sum_profit']['buckets'][0][result]['value'])
                else:
                    data[car][result]+=0
        
        # write the data
        
        with open("csv/data.pickle",'wb') as data_file_obj:
            pickle.dump(data,data_file_obj)   
    
        last_batch_processing_time+=time_delta
    else:        
        time.sleep(sleep_time)